In [1]:
%uv pip install -U accelerate bitsandbytes

Using Python 3.12.6 environment at: /usr/local
Resolved 47 packages in 218ms
⠙ Preparing packages... (0/40)
⠙ Preparing packages... (0/40)
⠙ Preparing packages... (0/40)
certifi    ------------------------------     0 B/149.32 KiB
⠙ Preparing packages... (0/40)
certifi    ------------------------------ 14.89 KiB/149.32 KiB
⠙ Preparing packages... (0/40)
anyio      ------------------------------     0 B/110.93 KiB
certifi    ------------------------------ 14.89 KiB/149.32 KiB
⠙ Preparing packages... (0/40)
click      ------------------------------     0 B/105.74 KiB
anyio      ------------------------------     0 B/110.93 KiB
certifi    ------------------------------ 14.89 KiB/149.32 KiB
⠙ Preparing packages... (0/40)
tqdm       ------------------------------     0 B/76.54 KiB
click      ------------------------------     0 B/105.74 KiB
anyio      ------------------------------     0 B/110.93 KiB
certifi    ------------------------------ 14.89 KiB/149.32 KiB
⠙ Preparing packages... (0/4

In [2]:
%uv pip install transformers==5.0.0

Using Python 3.12.6 environment at: /usr/local
Resolved 22 packages in 109ms
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 16.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 16.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ------------------------------ 32.00 KiB/3.12 MiB
⠙ Preparing packages... (0/2)
tokenizers ---

In [3]:
import transformers
print(transformers.__version__)

5.0.0


In [4]:
%uv pip install --upgrade torch torchvision

Using Python 3.12.6 environment at: /usr/local
Resolved 31 packages in 77ms
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
pillow     ------------------------------     0 B/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 16.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 32.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 32.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 32.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 32.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 32.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     ------------------------------ 32.00 KiB/6.72 MiB
⠙ Preparing packages... (0/2)
pillow     -------

In [7]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

# Cleaning khusus
def normalize_caption_post(text):
    if pd.isna(text):
        return None
    text = str(text).lower()
    text = re.sub(r"[—–−]+", "-", text)          
    text = re.sub(r"#\w+", "", text)              
    text = re.sub(r"http\S+|www\S+", "", text)    
    text = re.sub(r"[\U00010000-\U0010FFFF]", "", text)  
    return text.strip()

def clean_caption_text_from_row(row):
    caption = normalize_caption_post(row["caption"])
    if caption is None:
        return None
    
    nama_tempat = str(row.get("Nama Tempat", "")).lower().strip()
    alamat     = str(row.get("Alamat", "")).lower().strip()
    
    lines = [l.strip() for l in caption.split("\n") if l.strip()]
    cleaned_lines = []
    
    for line in lines:
        low = line.lower()
        if nama_tempat and nama_tempat in low:
            continue
        if alamat and alamat in low:
            continue
        
        
        line = re.sub(r"@\w+", "", line)          
        line = re.sub(r"#\w+", "", line)           
        line = re.sub(r"[^a-z\s.,!?]", " ", line)  
        line = re.sub(r"\s+", " ", line).strip()
        
        if line:
            cleaned_lines.append(line)
    
    return " ".join(cleaned_lines) if cleaned_lines else None

data = pd.read_csv("/root/lda_categorized_posts.csv")


print("Kolom yang tersedia:", data.columns.tolist())

Kolom yang tersedia: ['caption', 'Nama Tempat', 'Alamat', 'Jam Operasional', 'Kota', 'Akun Instagram', 'Jam Buka', 'Jam Tutup', 'Is 24H', 'Is Overnight', 'Kategori Tempat', 'Kategori Multilabel List', 'Kategori Multilabel']


In [8]:
data

,caption,Nama Tempat,Alamat,Jam Operasional,Kota,Akun Instagram,Jam Buka,Jam Tutup,Is 24H,Is Overnight,Kategori Tempat,Kategori Multilabel List,Kategori Multilabel
0,Seriussn waktu itu pas lewat kirain gedung nik...,Lyon’s Cafe & Coworking Space,"Jl. Nyi Tjondrolukito No 82, Kutu Dukuh, Sindu...",24 Jam,Sleman,@lyonscafe.co,00:00,23:59,True,False,Ngopi Santai,"['Ngopi Santai', 'WFC Nyaman']","Ngopi Santai, WFC Nyaman"
1,Seriusan coffee shop baru 24 jam dari Bandung ...,Beri Kopi Jogja,"Dero, Condongcatur, Sleman, Yogyakarta",24 Jam,Sleman,@berikopi.jogja,00:00,23:59,True,False,Ngopi Santai,"['Ngopi Santai', 'Area Lengkap']","Ngopi Santai, Area Lengkap"
2,Baru di Jogja - Artisan bakery & coffee esteti...,Elder On The Table - EOTB,"Jl. Kranggan, Jetis, Yogyakarta",07:00 - 22:00,Yogyakarta,"@elder.onthetable, @elderpatisserie",07:00,22:00,False,False,Ngopi Santai,['Ngopi Santai'],Ngopi Santai
3,Cold ‘N Brew baru aja buka cabangnya di Maliob...,Seperti cabang lainnya,"Jl. Mataram, Yogyakarta",24 Jam,Yogyakarta,@coldnbrew,00:00,23:59,True,False,Area Lengkap,['Area Lengkap'],Area Lengkap
4,Baru lagi di Jogja! Coffee & dining artistik a...,Waktu Luang De Loji,"Jl. Panembahan Senopati, Prawirodirjan, Kec. G...",10:00 - 22:00,Yogyakarta,@waktuluang.deloji,10:00,22:00,False,False,Ngopi Santai,['Ngopi Santai'],Ngopi Santai
...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,"Lunaria, coffee and eatery estetik baru di Ged...",Lunaria Coffee,Jl. Gedongkiwo No.29 (Pojok Beteng Kulon),08:00 - 22:00,Lainnya,@lunariacoffee,08:00,22:00,False,False,WFC Nyaman,"['Menu Variatif', 'WFC Nyaman']","Menu Variatif, WFC Nyaman"
305,"Kolokial Coffee, coffee shop bertema medieval ...",Kolokial Coffee,"Jl. Affandi No.30, Condongcatur, Depok, Kabupa...",09:00-22.00,Sleman,@kolokial.coffeeyk,09:00,22:00,False,False,Menu Variatif,"['Menu Variatif', 'WFC Nyaman']","Menu Variatif, WFC Nyaman"
306,Merindu bersama di Ruang Rindu Coffee\n\nCoffe...,Ruang Rindu Coffee,"Jl. Perumnas, Kledokan, Caturtunggal, Depok, S...",08:00 - 01:00,Sleman,@kopiruangrindu,08:00,01:00,False,True,WFC Nyaman,"['Menu Variatif', 'WFC Nyaman']","Menu Variatif, WFC Nyaman"
307,"Kastem Space Gantara, penyegaran coffee shop d...",Kasteman Space Gantara,"Jl. Lowanu, Sorosutan, Umbulharjo, Kota Yogyak...",10:00 - 24.00,Yogyakarta,@kastemspace_gantara,10:00,23:59,False,False,Menu Variatif,['Menu Variatif'],Menu Variatif


In [9]:
data['normalized_caption'] = data['caption'].apply(normalize_caption_post)

data['clean_caption'] = data.apply(clean_caption_text_from_row, axis=1)

data_clean = data[data['clean_caption'].notna() & (data['clean_caption'].str.strip() != '')].copy()
print(f"Jumlah data setelah cleaning: {len(data_clean)} (dari total {len(data)})")

model_id = "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config
)


def build_prompt(caption):
    return f"""\
Tugas:
Ekstrak informasi dari caption Instagram tentang coffee shop.

Aturan:
- Gunakan hanya informasi yang tertulis di caption.
- Jangan menambahkan informasi di luar caption.

Output:
1) Deskripsi
   - Apa yang dibahas (makanan, minuman, tempat, atau aktivitas di coffee shop).
   - Tulis jam operasionalnya.
   - Maksimal 1 kalimat saja.

2) Opini
   - Opini tentang makanan, minuman, tempat, atau suasana.
   - Opini boleh eksplisit atau implisit.
   - Jika tidak ada opini tersebut → "Netral".

Format wajib:

Deskripsi: <isi>
Opini: <isi>

Caption:
"{caption}"
"""

def extract_info(caption, max_new_tokens=150):
    prompt = build_prompt(caption)
    messages = [{"role": "user", "content": prompt}]
    
    encodings = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    
    input_ids = encodings.input_ids.to(model.device)
    attention_mask = encodings.attention_mask.to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,  
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def parse_llm_output(text):
    desc = "Tidak tersedia"
    opin = "Netral"
    
    if "Deskripsi:" in text:
        after_desc = text.split("Deskripsi:")[-1].strip()
        if "Opini:" in after_desc:
            d_part, o_part = after_desc.split("Opini:", 1)
            desc = d_part.strip()
            opin = o_part.strip()
        else:
            desc = after_desc.strip()
    
    desc = re.sub(r'\s+', ' ', desc).strip()
    opin = re.sub(r'\s+', ' ', opin).strip()
    
    if not desc or len(desc) < 5:
        desc = "Tidak tersedia"
    if not opin or len(opin) < 3:
        opin = "Netral"
    
    return desc, opin


deskripsi_list = []
opini_list = []

for caption in tqdm(data_clean['clean_caption'], desc="Ekstraksi dengan LLM"):
    if not caption.strip():
        deskripsi_list.append("Tidak tersedia")
        opini_list.append("Netral")
        continue
    
    raw = extract_info(caption)
    d, o = parse_llm_output(raw)
    deskripsi_list.append(d)
    opini_list.append(o)

data_clean['deskripsi'] = deskripsi_list
data_clean['opini'] = opini_list


data_clean.to_csv("/root/extracted_data_sahabatai(gemma).csv", index=False)

Jumlah data setelah cleaning: 309 (dari total 309)


Loading weights:   0%|          | 0/464 [00:00<?, ?it/s]

Ekstraksi dengan LLM: 100%|██████████████████████████████████████████████████| 309/309 [37:31<00:00,  7.29s/it]


In [10]:
data_clean

,caption,Nama Tempat,Alamat,Jam Operasional,Kota,Akun Instagram,Jam Buka,Jam Tutup,Is 24H,Is Overnight,Kategori Tempat,Kategori Multilabel List,Kategori Multilabel,normalized_caption,clean_caption,deskripsi,opini
0,Seriussn waktu itu pas lewat kirain gedung nik...,Lyon’s Cafe & Coworking Space,"Jl. Nyi Tjondrolukito No 82, Kutu Dukuh, Sindu...",24 Jam,Sleman,@lyonscafe.co,00:00,23:59,True,False,Ngopi Santai,"['Ngopi Santai', 'WFC Nyaman']","Ngopi Santai, WFC Nyaman",seriussn waktu itu pas lewat kirain gedung nik...,seriussn waktu itu pas lewat kirain gedung nik...,"Coffee shop luas dengan area parkir yang luas,...",Positif (menguji tempat duduk dan menu yang te...
1,Seriusan coffee shop baru 24 jam dari Bandung ...,Beri Kopi Jogja,"Dero, Condongcatur, Sleman, Yogyakarta",24 Jam,Sleman,@berikopi.jogja,00:00,23:59,True,False,Ngopi Santai,"['Ngopi Santai', 'Area Lengkap']","Ngopi Santai, Area Lengkap",seriusan coffee shop baru 24 jam dari bandung ...,seriusan coffee shop baru jam dari bandung ini...,Coffee shop baru di Bandung dengan menu yang e...,Tempat dan menu enak.
2,Baru di Jogja - Artisan bakery & coffee esteti...,Elder On The Table - EOTB,"Jl. Kranggan, Jetis, Yogyakarta",07:00 - 22:00,Yogyakarta,"@elder.onthetable, @elderpatisserie",07:00,22:00,False,False,Ngopi Santai,['Ngopi Santai'],Ngopi Santai,baru di jogja - artisan bakery & coffee esteti...,baru di jogja artisan bakery coffee estetik lu...,.onthetable adalah coffee shop yang baru buka ...,Tempat ini memiliki suasana yang nyaman dan me...
3,Cold ‘N Brew baru aja buka cabangnya di Maliob...,Seperti cabang lainnya,"Jl. Mataram, Yogyakarta",24 Jam,Yogyakarta,@coldnbrew,00:00,23:59,True,False,Area Lengkap,['Area Lengkap'],Area Lengkap,cold ‘n brew baru aja buka cabangnya di maliob...,cold n brew baru aja buka cabangnya di maliobo...,"Cold N Brew baru buka cabang di Malioboro, buk...",Netral.
4,Baru lagi di Jogja! Coffee & dining artistik a...,Waktu Luang De Loji,"Jl. Panembahan Senopati, Prawirodirjan, Kec. G...",10:00 - 22:00,Yogyakarta,@waktuluang.deloji,10:00,22:00,False,False,Ngopi Santai,['Ngopi Santai'],Ngopi Santai,baru lagi di jogja! coffee & dining artistik a...,baru lagi di jogja! coffee dining artistik aes...,"Coffee shop bernama .deloji di Jogja, dekat ti...","Positif, penulis memuji konsep, menu, dan pela..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,"Lunaria, coffee and eatery estetik baru di Ged...",Lunaria Coffee,Jl. Gedongkiwo No.29 (Pojok Beteng Kulon),08:00 - 22:00,Lainnya,@lunariacoffee,08:00,22:00,False,False,WFC Nyaman,"['Menu Variatif', 'WFC Nyaman']","Menu Variatif, WFC Nyaman","lunaria, coffee and eatery estetik baru di ged...","lunaria, coffee and eatery estetik baru di ged...","Lunaria, coffee and eatery estetik baru di Ged...","Suasana kondusif dan tenang, didukung oleh ker..."
305,"Kolokial Coffee, coffee shop bertema medieval ...",Kolokial Coffee,"Jl. Affandi No.30, Condongcatur, Depok, Kabupa...",09:00-22.00,Sleman,@kolokial.coffeeyk,09:00,22:00,False,False,Menu Variatif,"['Menu Variatif', 'WFC Nyaman']","Menu Variatif, WFC Nyaman","kolokial coffee, coffee shop bertema medieval ...",tempat ini sudah membuat kami penasaran sejak ...,Tempat nongkrong baru di pinggir jalan.,Netral.
306,Merindu bersama di Ruang Rindu Coffee\n\nCoffe...,Ruang Rindu Coffee,"Jl. Perumnas, Kledokan, Caturtunggal, Depok, S...",08:00 - 01:00,Sleman,@kopiruangrindu,08:00,01:00,False,True,WFC Nyaman,"['Menu Variatif', 'WFC Nyaman']","Menu Variatif, WFC Nyaman",merindu bersama di ruang rindu coffee\n\ncoffe...,coffee shop yang letaknya di jalan perumnas in...,Coffee shop dengan konsep coffee bar bertema g...,"Suasana coffee shop ini santai dan nostalgic, ..."
307,"Kastem Space Gantara, penyegaran coffee shop d...",Kasteman Space Gantara,"Jl. Lowanu, Sorosutan, Umbulharjo, Kota Yogyak...",10:00 - 24.00,Yogyakarta,@kastemspace_gantara,10:00,23:59,False,False,Menu Variatif,['Menu Variatif'],Menu Variatif,"kastem space gantara, penyegaran

In [5]:
%uv pip install openpyxl

Using Python 3.12.6 environment at: /usr/local
Resolved 2 packages in 33ms
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
et-xmlfile ------------------------------ 14.89 KiB/17.64 KiB
⠙ Preparing packages... (0/2)
et-xmlfile ------------------------------ 14.89 KiB/17.64 KiB
⠙ Preparing packages... (0/2)
et-xmlfile ------------------------------ 17.64 KiB/17.64 KiB
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
⠙ Preparing packages... (0/2)
Prepared 2 packages in 31ms
Installed 2 packages in 60ms
 + et-xmlfile==2.0.0
 + openpyxl==3.1.5
Note: you may need to restart the kernel to use updated packages.


In [13]:
data_clean.to_excel("/root/extracted_data_sahabatai.xlsx", index=False)
data_clean.to_csv("/root/extracted_data_sahabatai.csv", index=False)

In [11]:
data_filtered = data_clean[['clean_caption', 'opini', 'deskripsi']]

data_filtered.to_csv("extracted_data_sahabatai_1.csv", index=False)